In [1]:
import os
from copy import deepcopy

from rectools.dataset import Dataset
from rectools.model_selection import TimeRangeSplitter
from rectools.models import PopularModel, RandomModel
from rectools.metrics import MAP, NDCG, Precision, Recall, MeanInvUserFreq, Serendipity

from dev_eval import calculate_metrics, read_kion_dataset, visualize

DATA_DIR = '../data'
KION_DIR = os.path.join(DATA_DIR, 'data_original')
INTERACTIONS_DATA = os.path.join(KION_DIR, 'interactions.csv')
USERS_DATA = os.path.join(KION_DIR, 'users.csv')
ITEMS_DATA = os.path.join(KION_DIR, 'items.csv')

# Data

Подгружаем датасет кион используя дополнительную функцию

In [2]:
INTERACTIONS, USERS, ITEMS = read_kion_dataset()

# Параметры тестирования функций

Модели: `rectools.models.RandomModel(random_state=32)`, `rectools.models.PopularModel()` с параметрами по умолчанию

In [3]:
models = {
    "random": RandomModel(random_state=32),
    "popular": PopularModel(),
}

Метрики: 2 ранжирующие (MAP, NDCG), 2 классификационные (precision, recall), 2 beyond-accuracy (novelty, serendipity). Считаем по порогам 1, 5, 10. MAP обязательно

In [4]:
metrics = {
    "MAP@1": MAP(k=1),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
    "NDCG@1": NDCG(k=1),
    "NDCG@5": NDCG(k=5),
    "NDCG@10": NDCG(k=10),
    "precision@1": Precision(k=1),
    "precision@5": Precision(k=5),
    "precision@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),
    "novelty@1": MeanInvUserFreq(k=1),
    "novelty@5": MeanInvUserFreq(k=5),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@1": Serendipity(k=1),
    "serendipity@5": Serendipity(k=5),
    "serendipity@10": Serendipity(k=10),
}

Сплиттер: `rectools.model_selection.TimeRangeSplitter`, 3 фолда для кросс-валидации по неделе, исключение холодных юзеров и айтемов и просмотренных айтемов

In [5]:
cv = TimeRangeSplitter(
    test_size="7D",  # по неделе
    n_splits=3,  # 3 фолда для кросс-валидации
    filter_already_seen=True,  # исключение просмотренных айтемов
    filter_cold_items=True,  # исключение холодных айтемов 
    filter_cold_users=True,  #  исключение холодных юзеров
)

Визуализация рекомендаций и историй просмотров для юзеров `[666262, 672861, 955527]`. Для айтемов обязательно отражаем названия, жанры и количество просмотров в датасете (как для айтемов из истории взаимодействий каждого юзера, так и для айтемов из его рекомендаций)

In [6]:
USERS_CHECK = [666262, 672861, 955527]
K_RECOS = 10

# Функция 1: метрики 

In [7]:
result_data = calculate_metrics(models,
                                metrics,
                                cv,
                                K_RECOS)
result_data

  0%|          | 0/3 [00:00<?, ?it/s]

=======================================================|| Model: random | Fold: 0=======================================================
Fit time: 0.0 sec.
Recommend time: 5.1 sec.
Metrics time: 2.01 sec.
=======================================================|| Model: popular | Fold: 0=======================================================
Fit time: 1.12 sec.
Recommend time: 3.28 sec.
Metrics time: 1.85 sec.
=======================================================|| Model: random | Fold: 1=======================================================
Fit time: 0.0 sec.
Recommend time: 5.34 sec.
Metrics time: 2.06 sec.
=======================================================|| Model: popular | Fold: 1=======================================================
Fit time: 1.28 sec.
Recommend time: 3.5 sec.
Metrics time: 2.03 sec.
=======================================================|| Model: random | Fold: 2=======================================================
Fit time: 0.0 sec.
Recommend time: 5.

precision@1            recall@1           precision@5            \
               mean       std      mean       std        mean       std   
model                                                                     
popular    0.076432  0.006826  0.042720  0.004366    0.052402  0.001618   
random     0.000221  0.000033  0.000072  0.000019    0.000202  0.000028   

         recall@5           precision@10            ...  novelty@5            \
             mean       std         mean       std  ...       mean       std   
model                                               ...                        
popular  0.137413  0.005346     0.033903  0.001443  ...   3.066979  0.012316   
random   0.000365  0.000084     0.000193  0.000019  ...  15.612989  0.019570   

        novelty@10           serendipity@1               serendipity@5  \
              mean       std          mean           std          mean   
model                                                                    
popular   3.713390  0.002076      0.000002  3.733454e-08      0.000003   
random   15.613009  0.019786      0.000006  2.109823e-06      0.000007   

                      serendipity@10                
                  std           mean           std  
model                                               
popular  1.262334e-07       0.000002  8.147679e-08  
random   5.111606e-07       0.000007  2.550754e-07  

[2 rows x 36 columns]

# Функция 2: визуализация

In [8]:
dataset = (INTERACTIONS, USERS, ITEMS)
user_list = USERS_CHECK
item_data = ['title', 'genres']
K_RECOS = 10

In [9]:
for model_name in models:
    print(f'==={model_name}===')
    #extract model and train
    model = deepcopy(models[model_name])
    dataset_for_train = Dataset.construct(INTERACTIONS.df)
    model.fit(dataset_for_train)
    visualize(model,
              dataset,
              user_list,
              item_data,
              K_RECOS=10,
              display=display)

===random===
Visual report
----------------------------------------------------------
User: 666262
Already watched films amount: 3


,item_id,datetime,weight,watched_pct,title,genres,views
0,7957,2021-05-12,2052.0,32.0,Последний викинг,"боевики, историческое, приключения",746
1,4785,2021-05-12,1946.0,28.0,Робин Гуд: Начало,"боевики, триллеры, приключения",485
2,12981,2021-05-14,10292.0,100.0,Томирис,"боевики, драмы, историческое, военные",10370


Recomended films amount: 10
(Amount of all films: 15706)


,item_id,score,rank,title,genres,views
0,10101,10,1,Возвращение Будулая,мелодрамы,99
1,619,9,2,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии",1
2,12618,8,3,Пропавшая грамота,"фэнтези, комедии",51
3,5967,7,4,Братья вне игры,"драмы, спорт",262
4,4041,6,5,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики",19
5,5701,5,6,Алые паруса: Новая история,"комедии, мелодрамы",4
6,9738,4,7,Женщина в беде 3,"детективы, мелодрамы",2
7,15247,3,8,Гордость и предубеждение,"драмы, мелодрамы",150
8,10004,2,9,Болванчики,"мультфильм, приключения, комедии",51
9,2816,1,10,Избави нас от лукавого,"ужасы, триллеры, детективы",1370


----------------------------------------------------------
User: 672861
Already watched films amount: 2


...

,item_id,score,rank,title,genres,views
0,10440,202457.0,1,Хрустальный,"триллеры, детективы",202457
1,15297,193123.0,2,Клиника счастья,"драмы, мелодрамы",193123
2,9728,132865.0,3,Гнев человеческий,"боевики, триллеры",132865
3,13865,122119.0,4,Девятаев,"драмы, военные, приключения",122119
4,4151,91167.0,5,Секреты семейной жизни,комедии,91167
5,3734,74803.0,6,Прабабушка легкого поведения,комедии,74803
6,2657,68581.0,7,Подслушано,"драмы, триллеры",68581
7,4880,55043.0,8,Афера,комедии,55043
8,142,45367.0,9,Маша,"драмы, триллеры",45367
9,6809,40372.0,10,Дуров,документальное,40372


----------------------------------------------------------
User: 955527
Already watched films amount: 4


,item_id,datetime,weight,watched_pct,title,genres,views
0,1183,2021-06-02,40.0,1.0,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1587
1,13371,2021-05-04,686.0,11.0,Пеле: Рождение легенды,"драмы, спорт, биография",945
2,4725,2021-06-02,255.0,4.0,Лобановский навсегда,"спорт, биография, документальное",683
3,1238,2021-06-02,556.0,7.0,Диего Марадона,"спорт, биография, документальное",691


Recomended films amount: 10
(Amount of all films: 15706)


,item_id,score,rank,title,genres,views
0,10440,202457.0,1,Хрустальный,"триллеры, детективы",202457
1,15297,193123.0,2,Клиника счастья,"драмы, мелодрамы",193123
2,9728,132865.0,3,Гнев человеческий,"боевики, триллеры",132865
3,13865,122119.0,4,Девятаев,"драмы, военные, приключения",122119
4,4151,91167.0,5,Секреты семейной жизни,комедии,91167
5,3734,74803.0,6,Прабабушка легкого поведения,комедии,74803
6,2657,68581.0,7,Подслушано,"драмы, триллеры",68581
7,4880,55043.0,8,Афера,комедии,55043
8,142,45367.0,9,Маша,"драмы, триллеры",45367
9,6809,40372.0,10,Дуров,документальное,40372
